<a href="https://colab.research.google.com/github/manasarthak/Feature-Extraction-and-Electrode-Selection-for-Electroencephalogram-Based-emotion-classification/blob/main/Parallel_Bi_LSTM_Based_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.utils import class_weight,shuffle

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#importing layers
from keras.layers import Input,LSTM,BatchNormalization,Dropout,Dense,Activation,concatenate
from keras.models import Model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
dataset=np.load('/content/drive/MyDrive/DEAP/extracted_features/'+'fea.npy',allow_pickle=True);
ArLabels=np.load('/content/drive/MyDrive/DEAP/arousal/'+'all_arousal_labels.npy',allow_pickle=True);
ValLabels=np.load('/content/drive/MyDrive/DEAP/valence/'+'all_valence_labels.npy',allow_pickle=True);

In [ ]:
def channel_accuracy(data, labels):
    # Model accuracy score with default hyperparameters i.e C=1, kernel=rbf, gamma=auto
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(data, labels, test_size=0.15, random_state=42)
    svc = SVC()
    svc.fit(Xtrain, Ytrain)
    Ypred = svc.predict(Xtest)
    return accuracy_score(Ytest, Ypred)

# Assuming dataset and labels are defined elsewhere
for i in range(32):
    d1 = dataset[:, i:i+1, :]
    d1 = d1.reshape(1280, 6400)
    acc = channel_accuracy(d1, labels)
    print("Channel accuracy of channel no. ", i, " is ", acc)


In [ ]:
def process_data(data, labels, selected_indices):
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.05, random_state=6)

    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
    x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

    x_train_selected = [x_train[:, i, :].reshape(x_train.shape[0], 1, x_train.shape[-1]) for i in selected_indices]
    x_test_selected = [x_test[:, i, :].reshape(x_test.shape[0], 1, x_test.shape[-1]) for i in selected_indices]

    return x_train_selected, x_test_selected

# Assuming dataset and labels are defined elsewhere
selected_indices = [0, 1, 2, 3, 7, 8, 9, 15, 17, 18, 21, 28]

x_train_selected, x_test_selected = process_data(dataset, labels, selected_indices)

# Now x_train_selected and x_test_selected contain the transformed data
# You can access them using x_train_selected[0], x_train_selected[1], etc.

In [ ]:
inputs = []
outputs = []

for i in range(len(selected_indices)):
    channel_input = Input(shape=(ip1, ip2))
    lstm = LSTM(lstm_no, activation=act1)(channel_input)
    b = BatchNormalization()(lstm)
    drop = Dropout(dr)(b)
    dense = Dense(dense1, activation=act2)(drop)
    dense = BatchNormalization()(dense)
    dense = Dropout(dr)(dense)

    inputs.append(channel_input)
    outputs.append(dense)

concatenated = concatenate(outputs, axis=-1)
x = Dense(12, activation='relu')(concatenated)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)
x = Dense(12, activation='tanh')(x)
x = BatchNormalization()(x)

output = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=output)
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

history = model.fit(x=x_train_selected, y=y_train, epochs=500, batch_size=128,
                    validation_data=(x_test_selected, y_test))
print("Highest accuracy: " + str(max(history.history['val_accuracy'])))


In [ ]:
ip1 = 1   # Example value for the first input dimension
ip2 = 24  # Example value for the second input dimension
bi_lstm_no = 24
dense1 = 24
act1 = 'relu'
act2 = 'relu'
dr = 0.2

In [ ]:
inputs = []
outputs = []

for i in range(len(selected_indices)):
    channel_input = Input(shape=(ip1, ip2))
    bilstm = Bidirectional(LSTM(bi_lstm_no, activation=act1))(channel_input)
    b = BatchNormalization()(bilstm)
    drop = Dropout(dr)(b)
    dense = Dense(dense1, activation=act2)(drop)
    dense = BatchNormalization()(dense)
    dense = Dropout(dr)(dense)

    inputs.append(channel_input)
    outputs.append(dense)

concatenated = concatenate(outputs, axis=-1)
x = Dense(12, activation='relu')(concatenated)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)
x = Dense(12, activation='tanh')(x)
x = BatchNormalization()(x)

output = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=output)
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

history = model.fit(x=x_train_selected, y=y_train, epochs=500, batch_size=128,
                    validation_data=(x_test_selected, y_test))
print("Highest accuracy: " + str(max(history.history['val_accuracy'])))

In [ ]:
plot_model(model, to_file='model_val_plot.png', show_shapes=True, show_layer_names=True)

Barycenters are loaded
